In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime
from inspect import currentframe, getframeinfo # show line number


## load data

In [2]:
Q1 = pd.read_csv('data/ALLFAM08_Q1.csv' )
Q2 = pd.read_csv('data/ALLFAM08_Q2.csv' )
Q3 = pd.read_csv('data/ALLFAM08_Q3.csv' )
Q4 = pd.read_csv('data/ALLFAM08_Q4.csv' )
AllMb = pd.read_csv('data/ALLFAM08_AllMb.csv' )
OriginData1to12 = pd.read_csv('data/OriginData1to12.csv', encoding = 'big5hkscs' )

In [7]:
OriginData1to12 = pd.read_csv('data/OriginData1to12.csv', encoding = 'big5hkscs' )

In [7]:
display(
    Q1.shape,
    Q1.head(5),
    Q1.tail(5),
    AllMb.shape,
    Q1.head(5),
    Q1.tail(5)
)

(808946, 9)

,TxID,MbID,gender_0F1M,age,date,time_date_minus20161231,time_sec,SUM_of_sells,SUM_of_amount
0,1175820170312075416540251422720,1010002836,0,2,2017-03-12,71,6162986,6,259
1,1175820170226075241005937852416,1010002836,0,2,2017-02-26,57,4953014,7,294
2,1602620170211213419473839063040,1010005476,0,3,2017-02-11,42,3706449,3,178
3,1602620170323222445806170669056,1010005476,0,3,2017-03-23,82,7165934,2,139
4,1602620170213213299183368273920,1010005476,0,3,2017-02-13,44,3879427,1,30


,TxID,MbID,gender_0F1M,age,date,time_date_minus20161231,time_sec,SUM_of_sells,SUM_of_amount
808941,1282220170131234133799910506496,1143990215,0,2,2017-01-31,31,2763946,1,59
808942,1545820170205214752218504232960,1143990615,1,2,2017-02-05,36,3188941,2,99
808943,1337520170131235062275136028672,1143990615,1,2,2017-01-31,31,2764216,3,95
808944,1194520170221222790474441424896,1143990615,1,2,2017-02-21,52,4573924,6,132
808945,1045120170220204650122784538624,1143990740,0,1,2017-02-20,51,4481047,3,85


(75393, 3)

,TxID,MbID,gender_0F1M,age,date,time_date_minus20161231,time_sec,SUM_of_sells,SUM_of_amount
0,1175820170312075416540251422720,1010002836,0,2,2017-03-12,71,6162986,6,259
1,1175820170226075241005937852416,1010002836,0,2,2017-02-26,57,4953014,7,294
2,1602620170211213419473839063040,1010005476,0,3,2017-02-11,42,3706449,3,178
3,1602620170323222445806170669056,1010005476,0,3,2017-03-23,82,7165934,2,139
4,1602620170213213299183368273920,1010005476,0,3,2017-02-13,44,3879427,1,30


,TxID,MbID,gender_0F1M,age,date,time_date_minus20161231,time_sec,SUM_of_sells,SUM_of_amount
808941,1282220170131234133799910506496,1143990215,0,2,2017-01-31,31,2763946,1,59
808942,1545820170205214752218504232960,1143990615,1,2,2017-02-05,36,3188941,2,99
808943,1337520170131235062275136028672,1143990615,1,2,2017-01-31,31,2764216,3,95
808944,1194520170221222790474441424896,1143990615,1,2,2017-02-21,52,4573924,6,132
808945,1045120170220204650122784538624,1143990740,0,1,2017-02-20,51,4481047,3,85


## preprocess function

In [7]:
def preprocess_X(Qdata, Q):
    TTTTT = datetime.datetime #繼承 #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    checkpoint = TTTTT.now()#開始時間 #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間

    """
    >Prepares input features .
    >Args:
    Qdata: A Pandas DataFrame expected to contain Qdata.
    Q: A string indicated to the quarter for computing "Breadth" 
    >Returns:
    A DataFrame that contains the features to be used for the model, including
    synthetic features.
    """
    Qdata = Qdata.copy()

    print(getframeinfo(currentframe()).lineno, TTTTT.now()-checkpoint) #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    checkpoint = TTTTT.now()#開始時間 #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間

    start = datetime.datetime.now()
    #move other quantile to the baseline 
    #e.g.Q2 first day(04/01) : 91 --> 1
    movedays = min( Qdata.time_date_minus20161231)-1
    Qdata["time_date_minus20161231"] = Qdata["time_date_minus20161231"]- movedays
    Qdata["time_sec"] = Qdata["time_sec"] - movedays*86400
    print(getframeinfo(currentframe()).lineno, TTTTT.now()-checkpoint) #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    checkpoint = TTTTT.now()#開始時間 #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    #------------------------------------------------------------------------------------------------#
    # Feature engineering
    X = pd.DataFrame()
    # FE1.基本資料+RFM 
    X["age"] = Qdata.groupby("MbID").apply(lambda Qdata:max(Qdata["age"]))
    X["gender_0F1M"] = Qdata.groupby("MbID").apply(lambda Qdata:max(Qdata["gender_0F1M"]))
    X["R"] = Qdata.groupby("MbID").apply(lambda Qdata:max(Qdata["time_date_minus20161231"]))
    X["R_sec"] = Qdata.groupby("MbID").apply(lambda Qdata:max(Qdata["time_sec"]))
    
    X["F"] = Qdata.groupby("MbID").apply(lambda Qdata:len(Qdata["TxID"]))
    #Clip Outlier
    clip_temp = X["F"].quantile(0.75) +(3*(X["F"].quantile(0.75) - X["F"].quantile(0.25)))
    X["F"].clip(0, clip_temp, inplace = True)
    X["F_sd"] = Qdata.groupby("MbID").apply(lambda Qdata:np.std(Qdata["time_date_minus20161231"]))
    
    X["M"] = Qdata.groupby("MbID").apply(lambda Qdata: sum(Qdata["SUM_of_amount"]))
    #Clip Outlier
    clip_temp = X["M"].quantile(0.75) +(3*(X["M"].quantile(0.75) - X["M"].quantile(0.25)))
    X["M"].clip(0, clip_temp, inplace = True)
    X["M_sd"] = Qdata.groupby("MbID").apply(lambda Qdata: np.std(Qdata["SUM_of_amount"]))
    

    print(getframeinfo(currentframe()).lineno, TTTTT.now()-checkpoint) #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    checkpoint = TTTTT.now()#開始時間 #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    #------------------------------------------------------------------------------------------------#
    # FE2.Breadth + count of weekday 
    
    #計算Breadth:會員該季購買多少種類的商品
    data_temp = OriginData1to12.loc[OriginData1to12["季分"] == Q, ["會員卡號","單品名稱"] ].copy()
    data_temp.columns = ["MbID","單品名稱"]
    X["Breadth"] = data_temp.groupby("MbID").apply(lambda Qdata: Qdata["單品名稱"].unique().shape[0])
    #Clip Outlier
    clip_temp = X["Breadth"].quantile(0.75) +(3*(X["Breadth"].quantile(0.75) - X["Breadth"].quantile(0.25)))
    X["Breadth"].clip(0, clip_temp, inplace = True)
    
    
    #計算 count of weekday :會員在星期幾分別購買多少商品
    temp = pd.DataFrame()
    temp["date"] = pd.to_datetime(Qdata.loc[:,"date"])
    temp['day_of_week'] = temp["date"].dt.dayofweek
    days = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
    temp['day_of_week'] = temp['day_of_week'].apply(lambda x: days[x])
    Qdata = pd.concat([Qdata, temp['day_of_week']], axis = 1 )
    #ite_weekday_temp= []
    X["CountSun"] = Qdata.groupby("MbID").apply(lambda Qdata: sum( Qdata["day_of_week"]=="Sun") )
    X["CountMon"] = Qdata.groupby("MbID").apply(lambda Qdata: sum( Qdata["day_of_week"]=="Mon") )
    X["CountTue"] = Qdata.groupby("MbID").apply(lambda Qdata: sum( Qdata["day_of_week"]=="Tue") )
    X["CountWed"] = Qdata.groupby("MbID").apply(lambda Qdata: sum( Qdata["day_of_week"]=="Wed") )
    X["CountThu"] = Qdata.groupby("MbID").apply(lambda Qdata: sum( Qdata["day_of_week"]=="Thu") )
    X["CountFri"] = Qdata.groupby("MbID").apply(lambda Qdata: sum( Qdata["day_of_week"]=="Fri") )
    X["CountSat"] = Qdata.groupby("MbID").apply(lambda Qdata: sum( Qdata["day_of_week"]=="Sat") )
    #Clip Outlier
    clip_temp = X["CountSun"].quantile(0.75) +(3*(X["CountSun"].quantile(0.75) - X["CountSun"].quantile(0.25)))
    X["CountSun"].clip(0, clip_temp, inplace = True)
    clip_temp = X["CountMon"].quantile(0.75) +(3*(X["CountMon"].quantile(0.75) - X["CountMon"].quantile(0.25)))
    X["CountMon"].clip(0, clip_temp, inplace = True)
    clip_temp = X["CountTue"].quantile(0.75) +(3*(X["CountTue"].quantile(0.75) - X["CountTue"].quantile(0.25)))
    X["CountTue"].clip(0, clip_temp, inplace = True)
    clip_temp = X["CountWed"].quantile(0.75) +(3*(X["CountWed"].quantile(0.75) - X["CountWed"].quantile(0.25)))
    X["CountWed"].clip(0, clip_temp, inplace = True)
    clip_temp = X["CountThu"].quantile(0.75) +(3*(X["CountThu"].quantile(0.75) - X["CountThu"].quantile(0.25)))
    X["CountThu"].clip(0, clip_temp, inplace = True)
    clip_temp = X["CountFri"].quantile(0.75) +(3*(X["CountFri"].quantile(0.75) - X["CountFri"].quantile(0.25)))
    X["CountFri"].clip(0, clip_temp, inplace = True)
    clip_temp = X["CountSat"].quantile(0.75) +(3*(X["CountSat"].quantile(0.75) - X["CountSat"].quantile(0.25)))
    X["CountSat"].clip(0, clip_temp, inplace = True)

    print(getframeinfo(currentframe()).lineno, TTTTT.now()-checkpoint) #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    checkpoint = TTTTT.now()#開始時間 #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    #------------------------------------------------------------------------------------------------#
    #FE3.Clumpiness 
    ##https://repository.upenn.edu/cgi/viewcontent.cgi?article=1081&context=statistics_papers

    ## compute ITEs For i=2,. . .,n, Xi = ti − ti−1; X1 = t1 
    Qlen = np.shape(Qdata)[0] 
    Qdata = Qdata.sort_values(
        by=["MbID", "time_date_minus20161231", "time_sec"] ).reset_index(drop=True)
    Qdata_ITE = Qdata[["MbID","time_date_minus20161231","time_sec"]].copy()

    temp = "0"
    ite_List_temp= []#創建一個List
    ite_List_sec_temp= []#創建一個List
    Qdata_ITE["ite"] = None
    for i in range (0, Qlen ) :
        if temp == Qdata.loc[i, "MbID"] :
            ite_List_temp.append( 
                Qdata.loc[i, "time_date_minus20161231"] 
                - Qdata.loc[i-1, "time_date_minus20161231"] )
            ite_List_sec_temp.append( 
                Qdata.loc[i, "time_sec"] 
                - Qdata.loc[i-1, "time_sec"] )
        else :
            ite_List_temp.append( Qdata.loc[i, "time_date_minus20161231"] ) 
            ite_List_sec_temp.append( Qdata.loc[i, "time_sec"] )
            temp = Qdata.loc[i, "MbID"]
        if i%10000 == 0 :
            print(i)


    Qdata_ITE.loc[:,"ite"] = None #創建欄位
    Qdata_ITE.loc[:,"ite_sec"] = None #創建欄位
    Qdata_ITE.loc[:,"ite"] = ite_List_temp #將list貼上該欄
    Qdata_ITE.loc[:,"ite_sec"] = ite_List_sec_temp #將list貼上該欄
    print(getframeinfo(currentframe()).lineno,TTTTT.now()-checkpoint) #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    checkpoint = TTTTT.now()#開始時間 #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    ## compute ITEs For  Xn+1 = N − tn. 
    X = X.reset_index(level="MbID", drop=False) # MbID was index from groupby, transform to column
    Qperiod = (max( Qdata.time_date_minus20161231)) - (min( Qdata.time_date_minus20161231))
    Qperiod_sec = (max( Qdata.time_sec)) - (min( Qdata.time_sec))
    X_temp = X.assign(Xnplus1 = lambda x : Qperiod + 1 -X.R )
    X_temp = X_temp.assign(Xnplus1_sec = lambda x : Qperiod + 1 -X.R_sec )
    X_temp = X_temp[["MbID","R","Xnplus1","Xnplus1_sec"]]
    X_temp.columns = ["MbID","time_date_minus20161231", "ite","ite_sec"]
    ## Xn+1 append to X1,....,Xn
    Qdata_ITE = Qdata_ITE.append(X_temp, ignore_index=True, sort=False)
    Qdata_ITE = Qdata_ITE.sort_values(
        by=["MbID", "time_date_minus20161231"] ).reset_index(drop=True)#""""""
    ## normalize  Xi
    Qdata_ITE = (Qdata_ITE - Qdata_ITE.min()) / (Qdata_ITE.max() - Qdata_ITE.min())
    print(getframeinfo(currentframe()).lineno, TTTTT.now()-checkpoint) #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    checkpoint = TTTTT.now()#開始時間 #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    #------------------------------------------------------------------------------------------------#
    ## clumpiness measures
    temp = pd.DataFrame()
    temp["L2"] = Qdata_ITE.groupby("MbID").apply(
        lambda Qdata_ITE : sum( Qdata_ITE["ite"]**2 ))
    #temp["L2_sec"] = Qdata_ITE.groupby("MbID").apply(lambda Qdata_ITE : sum( Qdata_ITE["ite_sec"]**2 ))
    temp["Entropy"] = Qdata_ITE.groupby("MbID").apply(  
        lambda Qdata_ITE : sum( np.log(Qdata_ITE["ite"]+0.0000001) * Qdata_ITE["ite"]))
    #temp["Entropy_sec"] = Qdata_ITE.groupby("MbID").apply(  lambda Qdata_ITE : sum( np.log(Qdata_ITE["ite_sec"]+0.0000001) * Qdata_ITE["ite_sec"]))
    temp["LogUtility"] = Qdata_ITE.groupby("MbID").apply(
        lambda Qdata_ITE : sum( -np.log(Qdata_ITE["ite"]+0.0000001)))
    #temp["LogUtility_sec"] = Qdata_ITE.groupby("MbID").apply(lambda Qdata_ITE : sum( -np.log(Qdata_ITE["ite_sec"]+0.0000001)))
    ###C3
    def sum_C3(Qdata_ITE):
        temp = Qdata_ITE["ite"]
        temp2 = temp.nlargest(3)
        if temp2.shape[0] <= 2 :
            temp3 = Qperiod + 1 
        else : 
            temp3 = sum( temp2 ) 
        return temp3
    temp["C3"] = Qdata_ITE.groupby("MbID").apply( sum_C3 )
    def sum_C3_sec(Qdata_ITE):
        temp = Qdata_ITE["ite_sec"]
        temp2 = temp.nlargest(3)
        if temp2.shape[0] <= 2 :
            temp3 = Qperiod + 1 
        else : 
            temp3 = sum( temp2 ) 
        return temp3
    #temp["C3_sec"] = Qdata_ITE.groupby("MbID").apply( sum_C3_sec )
    print(getframeinfo(currentframe()).lineno, TTTTT.now()-checkpoint) #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    checkpoint = TTTTT.now()#開始時間 #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    #------------------------------------------------------------------------------------------------#
    #bind all the X features
    X = pd.concat([X, temp.reset_index(level="MbID", drop=True)], axis=1, sort=False )    

    # normalization 
    temp = X.drop(["MbID","age","gender_0F1M"], axis=1).copy()
    temp = (temp - temp.min()) / (temp.max() - temp.min())
    X = pd.concat([X[["MbID","age","gender_0F1M"]].copy(),temp], axis=1, sort=False) 
    # NaN補0，以免出現0/0=NaN
    X.fillna(value = 0, inplace=True)
    print(getframeinfo(currentframe()).lineno, TTTTT.now()-checkpoint) #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    checkpoint = TTTTT.now()#開始時間 #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    #------------------------------------------------------------------------------------------------#
    # Customers who didn't show in the quarter
    temp = pd.DataFrame(columns=['MbID', 'age', 'gender_0F1M', 'R', 'R_sec', 'F', 'F_sd', 'M', 'M_sd',
       'L2', 'Entropy', 'LogUtility', 'C3'])
    NoShowMb = AllMb[~ AllMb.MbID.isin(Qdata.MbID)]
    temp = pd.DataFrame(data={'MbID':NoShowMb["MbID"],
                              "age":NoShowMb["age"],
                              "gender_0F1M":NoShowMb["gender"],
                              "R":np.ones(NoShowMb["MbID"].shape[0]),
                              "R_sec":np.ones(NoShowMb["MbID"].shape[0]),
                              "F":np.zeros(NoShowMb["MbID"].shape[0]),
                              "F_sd":np.zeros(NoShowMb["MbID"].shape[0]),
                              "M":np.zeros(NoShowMb["MbID"].shape[0]),
                              "M_sd":np.zeros(NoShowMb["MbID"].shape[0]),
                              "Breadth":np.zeros(NoShowMb["MbID"].shape[0]),
                              "CountSun":np.zeros(NoShowMb["MbID"].shape[0]),
                              "CountMon":np.zeros(NoShowMb["MbID"].shape[0]),
                              "CountTue":np.zeros(NoShowMb["MbID"].shape[0]),
                              "CountWed":np.zeros(NoShowMb["MbID"].shape[0]),
                              "CountThu":np.zeros(NoShowMb["MbID"].shape[0]),
                              "CountFri":np.zeros(NoShowMb["MbID"].shape[0]),
                              "CountSat":np.zeros(NoShowMb["MbID"].shape[0]),
                              "L2":np.ones(NoShowMb["MbID"].shape[0]),
                              "Entropy":np.ones(NoShowMb["MbID"].shape[0]),
                              "LogUtility":np.ones(NoShowMb["MbID"].shape[0]),
                              "C3":np.ones(NoShowMb["MbID"].shape[0])
                             })
    print(getframeinfo(currentframe()).lineno, TTTTT.now()-checkpoint) #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    checkpoint = TTTTT.now()#開始時間 #時間時間時間時間時間時間時間時間時間時間時間時間時間時間時間
    #------------------------------------------------------------------------------------------------#
    X = X.append(temp, sort=False).sort_values(by=["MbID"]).reset_index(drop=True)

    return X

In [8]:
Q1_X = preprocess_X(Q1.iloc[:1000,],Q = "Q1")
display(
    Q1_X.shape,
    Q1_X.head(10)
)
Q1_X.to_csv("Q1_X_clipped.csv",index=False)

16 0:00:00.007011
25 0:00:00.003992
49 0:00:00.043877
94 0:00:05.911612
0
130 0:00:00.031209
146 0:00:00.015649
179 0:00:00.046868
191 0:00:00
220 0:00:00.015618


(75393, 21)

,MbID,age,gender_0F1M,R,R_sec,F,F_sd,M,M_sd,Breadth,...,CountMon,CountTue,CountWed,CountThu,CountFri,CountSat,L2,Entropy,LogUtility,C3
0,1010002836,2,0,0.784091,0.779552,0.010204,0.198722,0.037927,0.018429,0.026144,...,0.00,0.0000,0.00,0.000000,0.000000,0.00,0.482541,0.804501,0.001001,0.010489
1,1010003550,0,0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0000,0.00,0.000000,0.000000,0.00,1.000000,1.000000,1.000000,1.000000
2,1010004795,2,0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0000,0.00,0.000000,0.000000,0.00,1.000000,1.000000,1.000000,1.000000
3,1010005476,3,0,0.977273,0.977687,0.214286,0.709563,0.124699,0.070898,0.124183,...,0.30,0.3125,0.20,0.142857,0.044444,0.10,0.074823,0.373222,0.049602,0.003665
4,1010005510,1,0,1.000000,0.999258,0.408163,0.741870,0.283087,0.104654,0.411765,...,0.55,0.3125,0.25,0.214286,0.444444,0.25,0.045944,0.257541,0.122049,0.002780
5,1010005565,3,0,0.852273,0.853773,0.030612,0.410106,0.019969,0.049682,0.058824,...,0.05,0.0000,0.05,0.000000,0.000000,0.05,0.269100,0.667998,0.003084,0.007582
6,1010006000,2,0,0.988636,0.980422,1.000000,0.662097,1.000000,0.152715,1.000000,...,1.00,1.0000,1.00,1.000000,0.711111,1.00,0.000525,0.002840,1.000000,0.000253
7,1010006466,2,0,0.965909,0.961736,0.010204,0.042583,0.003951,0.017903,0.006536,...,0.00,0.0625,0.00,0.000000,0.000000,0.05,0.911525,0.954924,0.002313,0.010489
8,1010006692,2,1,1.000000,0.998747,0.520408,0.728982,0.248034,0.055015,0.568627,...,0.60,0.1875,0.30,0.785714,0.355556,0.45,0.044369,0.228316,0.170231,0.002780
9,1010007187,2,1,1.000000,0.995663,0.551020,0.677660,0.373451,0.181106,0.346405,...,0.30,0.4375,0.45,0.928571,0.444444,0.20,0.016540,0.137975,0.150054,0.001137


In [17]:
def preprocess_Y(Qdata):
    """
    >Prepares target features (i.e., labels) from California housing data set.
    >Args:
    Qdata: A Pandas DataFrame expected to contain data
    >Returns:
    A DataFrame that contains the target feature.
    """
    Qdata = Qdata.copy()
    #move other quantile to the baseline 
    #e.g.Q2 first day(04/01) : 91 --> 1
    movedays = min( Qdata.time_date_minus20161231)-1
    Qdata["time_date_minus20161231"] = Qdata["time_date_minus20161231"]- movedays
    Qdata["time_sec"] = Qdata["time_sec"] - movedays*86400
    
    # Feature engineering
    movedays = min( Qdata.time_date_minus20161231)-1
    Y = pd.DataFrame()

    Y["R"] = Qdata.groupby("MbID").apply(lambda Qdata:max(Qdata["time_date_minus20161231"]))
    maxR=max(Y["R"]) # 找出最大的R替換NoShowMb的空值
    Y["R_sec"] = Qdata.groupby("MbID").apply(lambda Qdata:max(Qdata["time_sec"]))
    maxR_sec=max(Y["R"]) # 找出最大的R替換NoShowMb的空值
    Y["F"] = Qdata.groupby("MbID").apply(lambda Qdata:len(Qdata["TxID"]))
    #Clip Outlier
    clip_temp = Y["F"].quantile(0.75) +(3*(Y["F"].quantile(0.75) - Y["F"].quantile(0.25)))
    Y["F_CLipped"] = Y["F"].clip(0, clip_temp, inplace = False)
    
    Y["M"] = Qdata.groupby("MbID").apply(lambda Qdata:sum(Qdata["SUM_of_amount"]))
    #Clip Outlier
    clip_temp = Y["M"].quantile(0.75) +(3*(Y["M"].quantile(0.75) - Y["M"].quantile(0.25)))
    Y["M_CLipped"] = Y["M"].clip(0, clip_temp, inplace = False)
    
    # Classification in terms of R F M
    Y["R_class01"] = Y["R"].apply(lambda R : 1 if R >= np.median(Y["R"]) else 0)
    Y["F_class01"] = Y["F"].apply(lambda R : 1 if R >= np.median(Y["F"]) else 0)
    Y["M_class01"] = Y["M"].apply(lambda R : 1 if R >= np.median(Y["M"]) else 0)
    
    Y["R_class012"] = np.where(Y["R"]>= np.percentile(Y["R"],1/3), 1,0)
    Y["R_class012"] = np.where(Y["R"]>= np.percentile(Y["R"],2/3), 2,1)
    
    Y["R_class0123"] = np.where(Y["R"]>= np.percentile(Y["R"],1/4), 1,0)
    Y["R_class0123"] = np.where(Y["R"]>= np.percentile(Y["R"],2/4), 2,1)
    Y["R_class0123"] = np.where(Y["R"]>= np.percentile(Y["R"],3/4), 3,2)
    
    Y["R_class01234"] = np.where(Y["R"]>= np.percentile(Y["R"],1/5), 1,0)
    Y["R_class01234"] = np.where(Y["R"]>= np.percentile(Y["R"],2/5), 2,1)
    Y["R_class01234"] = np.where(Y["R"]>= np.percentile(Y["R"],3/5), 3,2)
    Y["R_class01234"] = np.where(Y["R"]>= np.percentile(Y["R"],4/5), 4,3)
    
    ## normalize  
    #Y = (Y - Y.min()) / (Y.max() - Y.min())
    
    # Reset index
    Y = Y.reset_index(level="MbID", drop=False)
    
    # Customers who didn't show in the quarter
    temp = pd.DataFrame(columns=['MbID', 'R', 'R_sec', 'F', 'M', 'R_class01','F_class01', 'M_class01', 
                                 'R_class012', 'R_class0123', 'R_class01234'])
    NoShowMb = AllMb[~ AllMb.MbID.isin(Qdata.MbID)]
    temp = pd.DataFrame(data={'MbID':NoShowMb["MbID"],
                              "R":np.full(NoShowMb["MbID"].shape[0], maxR),   ###改成最大天數
                              "R_sec":np.full(NoShowMb["MbID"].shape[0], maxR_sec),
                              "F":np.zeros(NoShowMb["MbID"].shape[0]),
                              "F_CLipped":np.zeros(NoShowMb["MbID"].shape[0]),
                              "M":np.zeros(NoShowMb["MbID"].shape[0]),
                              "M_CLipped":np.zeros(NoShowMb["MbID"].shape[0]),
                              "R_class01":np.zeros(NoShowMb["MbID"].shape[0]),
                              "F_class01":np.zeros(NoShowMb["MbID"].shape[0]),
                              "M_class01":np.zeros(NoShowMb["MbID"].shape[0]),
                              "R_class012":np.zeros(NoShowMb["MbID"].shape[0]),
                              "R_class0123":np.zeros(NoShowMb["MbID"].shape[0]),
                              "R_class01234":np.zeros(NoShowMb["MbID"].shape[0])
                             })
    Y = Y.append(temp, sort=False).sort_values(by=["MbID"]).reset_index(drop=True)
    
    return Y


In [9]:
Q1_X = preprocess_X(Q1.iloc[:,:], Q = 'Q1')
display(
    Q1_X.shape,
    Q1_X.head(10)
)
Q1_X.to_csv("Q1_X_Clipped.csv",index=False)

16 0:00:00.038896
25 0:00:00.034247
49 0:00:27.214024
94 0:00:59.309515
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
130 0:00:28.272350
146 0:00:00.650832
179 0:00:46.333362
191 0:00:00.078089
220 0:00:00.031242


(75393, 21)

,MbID,age,gender_0F1M,R,R_sec,F,F_sd,M,M_sd,Breadth,...,CountMon,CountTue,CountWed,CountThu,CountFri,CountSat,L2,Entropy,LogUtility,C3
0,1010002836,2,0,0.786517,0.781447,0.018868,0.157303,0.084637,0.000137,0.048193,...,0.000,0.000,0.000,0.000,0.000,0.000,0.463920,0.798416,0.000073,0.010745
1,1010003550,0,0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000,0.000,0.000,0.000,0.000,1.000000,1.000000,1.000000,1.000000
2,1010004795,2,0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000,0.000,0.000,0.000,0.000,1.000000,1.000000,1.000000,1.000000
3,1010005476,3,0,0.977528,0.975192,0.396226,0.561672,0.269856,0.000526,0.228916,...,0.750,0.625,0.500,0.250,0.125,0.250,0.076155,0.405957,0.003360,0.004076
4,1010005510,1,0,1.000000,0.996285,0.754717,0.587245,0.607942,0.000777,0.759036,...,1.000,0.625,0.625,0.375,1.000,0.625,0.048689,0.300690,0.008252,0.003211
5,1010005565,3,0,0.853933,0.854024,0.056604,0.324630,0.046305,0.000369,0.108434,...,0.125,0.000,0.125,0.000,0.000,0.125,0.260924,0.674200,0.000215,0.007904
6,1010006000,2,0,0.988764,0.977867,1.000000,0.524099,1.000000,0.001133,1.000000,...,1.000,1.000,1.000,1.000,1.000,1.000,0.005493,0.068915,0.067454,0.000741
7,1010006466,2,0,0.966292,0.959595,0.018868,0.033708,0.012113,0.000133,0.012048,...,0.000,0.125,0.000,0.000,0.000,0.125,0.871910,0.935299,0.000162,0.010745
8,1010006692,2,1,1.000000,0.995786,0.962264,0.577044,0.533119,0.000408,1.000000,...,1.000,0.375,0.750,1.000,1.000,1.000,0.047191,0.274095,0.011503,0.003211
9,1010007187,2,1,1.000000,0.992770,1.000000,0.536419,0.800828,0.001344,0.638554,...,0.750,0.875,1.000,1.000,1.000,0.500,0.020724,0.191886,0.010148,0.001606


In [10]:
Q2_X = preprocess_X(Q2.iloc[:,:], Q = 'Q2')
display(
    Q2_X.shape,
    Q2_X.head(10)
)
Q2_X.to_csv("Q2_X_Clipped.csv",index=False)

16 0:00:00.041888
25 0:00:00.048870
49 0:00:26.342862
94 0:00:56.329516
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
130 0:00:30.484638
146 0:00:00.693771
179 0:00:43.891298
191 0:00:00.062484
220 0:00:00.031247


(75393, 21)

,MbID,age,gender_0F1M,R,R_sec,F,F_sd,M,M_sd,Breadth,...,CountMon,CountTue,CountWed,CountThu,CountFri,CountSat,L2,Entropy,LogUtility,C3
0,1010002836,2,0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,1.000000,1.000000,1.000000,1.000000
1,1010003550,0,0,0.655556,0.655779,0.076923,0.235877,0.041917,0.000142,0.073684,...,0.000000,0.166667,0.000000,0.083333,0.000000,0.125,0.275702,0.687693,0.001017,0.009664
2,1010004795,2,0,0.888889,0.882574,0.276923,0.309595,0.224320,0.000154,0.105263,...,0.166667,0.250000,0.333333,0.250000,0.583333,0.000,0.162882,0.494997,0.004712,0.006040
3,1010005476,3,0,0.944444,0.943968,0.492308,0.569029,0.405866,0.000416,0.463158,...,0.416667,0.416667,0.750000,0.166667,0.416667,0.625,0.062027,0.351613,0.012635,0.003624
4,1010005510,1,0,0.611111,0.609463,0.061538,0.266889,0.048069,0.000278,0.084211,...,0.166667,0.000000,0.000000,0.000000,0.083333,0.250,0.297436,0.713675,0.001444,0.009906
5,1010005565,3,0,0.977778,0.970822,0.153846,0.348671,0.259084,0.000732,0.147368,...,0.083333,0.166667,0.083333,0.166667,0.166667,0.125,0.210745,0.578271,0.002054,0.007007
6,1010006000,2,0,1.000000,0.998794,1.000000,0.606680,1.000000,0.001466,0.905263,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000,0.011477,0.125849,0.055591,0.001087
7,1010006466,2,0,1.000000,0.991803,0.753846,0.566838,0.344635,0.000208,0.326316,...,0.416667,0.666667,0.916667,0.833333,1.000000,0.250,0.025641,0.219119,0.017115,0.001812
8,1010006692,2,1,0.988889,0.987074,0.784615,0.595701,0.511588,0.000334,0.800000,...,0.666667,0.833333,0.833333,0.833333,0.416667,0.875,0.033455,0.245560,0.020361,0.002416
9,1010007187,2,1,1.000000,0.992791,1.000000,0.575834,0.720029,0.000291,0.968421,...,0.916667,1.000000,1.000000,1.000000,1.000000,0.625,0.013187,0.137871,0.046757,0.001208


In [11]:
Q3_X = preprocess_X(Q2.iloc[:,:], Q = 'Q3')
display(
    Q3_X.shape,
    Q3_X.head(10)
)
Q3_X.to_csv("Q3_X_Clipped.csv",index=False)

16 0:00:00.033941
25 0:00:00.038895
49 0:00:25.908008
94 0:00:56.283961
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
130 0:00:30.606335
146 0:00:00.717917
179 0:00:43.650953
191 0:00:00.077793
220 0:00:00.024931


(75393, 21)

,MbID,age,gender_0F1M,R,R_sec,F,F_sd,M,M_sd,Breadth,...,CountMon,CountTue,CountWed,CountThu,CountFri,CountSat,L2,Entropy,LogUtility,C3
0,1010002836,2,0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,1.000000,1.000000,1.000000,1.000000
1,1010003550,0,0,0.655556,0.655779,0.076923,0.235877,0.041917,0.000142,0.000000,...,0.000000,0.166667,0.000000,0.083333,0.000000,0.125,0.275702,0.687693,0.001017,0.009664
2,1010004795,2,0,0.888889,0.882574,0.276923,0.309595,0.224320,0.000154,0.135593,...,0.166667,0.250000,0.333333,0.250000,0.583333,0.000,0.162882,0.494997,0.004712,0.006040
3,1010005476,3,0,0.944444,0.943968,0.492308,0.569029,0.405866,0.000416,0.389831,...,0.416667,0.416667,0.750000,0.166667,0.416667,0.625,0.062027,0.351613,0.012635,0.003624
4,1010005510,1,0,0.611111,0.609463,0.061538,0.266889,0.048069,0.000278,0.000000,...,0.166667,0.000000,0.000000,0.000000,0.083333,0.250,0.297436,0.713675,0.001444,0.009906
5,1010005565,3,0,0.977778,0.970822,0.153846,0.348671,0.259084,0.000732,0.033898,...,0.083333,0.166667,0.083333,0.166667,0.166667,0.125,0.210745,0.578271,0.002054,0.007007
6,1010006000,2,0,1.000000,0.998794,1.000000,0.606680,1.000000,0.001466,0.559322,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000,0.011477,0.125849,0.055591,0.001087
7,1010006466,2,0,1.000000,0.991803,0.753846,0.566838,0.344635,0.000208,0.271186,...,0.416667,0.666667,0.916667,0.833333,1.000000,0.250,0.025641,0.219119,0.017115,0.001812
8,1010006692,2,1,0.988889,0.987074,0.784615,0.595701,0.511588,0.000334,0.449153,...,0.666667,0.833333,0.833333,0.833333,0.416667,0.875,0.033455,0.245560,0.020361,0.002416
9,1010007187,2,1,1.000000,0.992791,1.000000,0.575834,0.720029,0.000291,0.805085,...,0.916667,1.000000,1.000000,1.000000,1.000000,0.625,0.013187,0.137871,0.046757,0.001208


In [18]:
Q2_Y = preprocess_Y(Q2.iloc[:,]) 
display(
    Q2_Y.shape,
    Q2_Y.head(5),
    Q2_Y.tail(5)
)
Q2_Y.to_csv("Q2_Y_Clipped.csv",index=False)

(75393, 13)

,MbID,R,R_sec,F,F_CLipped,M,M_CLipped,R_class01,F_class01,M_class01,R_class012,R_class0123,R_class01234
0,1010002836,91,91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1010003550,60,5243213,6.0,6.0,293.0,293.0,0.0,1.0,0.0,2.0,3.0,4.0
2,1010004795,81,7025805,19.0,19.0,1568.0,1568.0,0.0,1.0,1.0,2.0,3.0,4.0
3,1010005476,86,7508351,33.0,33.0,2837.0,2837.0,1.0,1.0,1.0,2.0,3.0,4.0
4,1010005510,56,4879174,5.0,5.0,336.0,336.0,0.0,0.0,0.0,2.0,3.0,4.0


,MbID,R,R_sec,F,F_CLipped,M,M_CLipped,R_class01,F_class01,M_class01,R_class012,R_class0123,R_class01234
75388,1143990295,91,91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75389,1143990552,63,5504850,3.0,3.0,411.0,411.0,0.0,0.0,0.0,2.0,3.0,4.0
75390,1143990615,91,91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75391,1143990740,91,91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75392,1143990826,24,2103555,5.0,5.0,385.0,385.0,0.0,0.0,0.0,2.0,3.0,4.0


In [19]:
Q3_Y = preprocess_Y(Q3.iloc[:,:]) 
display(
    Q3_Y.shape,
    Q3_Y.head(5)
)
Q3_Y.to_csv("Q3_Y_Clipped.csv",index=False)

(75393, 13)

,MbID,R,R_sec,F,F_CLipped,M,M_CLipped,R_class01,F_class01,M_class01,R_class012,R_class0123,R_class01234
0,1010002836,92,92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1010003550,92,92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1010004795,91,7889382,17.0,17.0,1739.0,1739.0,1.0,1.0,1.0,2.0,3.0,4.0
3,1010005476,88,7683710,32.0,32.0,3805.0,3805.0,1.0,1.0,1.0,2.0,3.0,4.0
4,1010005510,92,92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
Q4_Y = preprocess_Y(Q4.iloc[:,:]) 
display(
    Q4_Y.shape,
    Q4_Y.head(5)
)
Q4_Y.to_csv("Q4_Y_Clipped.csv",index=False)

(75393, 13)

,MbID,R,R_sec,F,F_CLipped,M,M_CLipped,R_class01,F_class01,M_class01,R_class012,R_class0123,R_class01234
0,1010002836,92,92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1010003550,92,92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1010004795,83,7199859,18.0,18.0,815.0,815.0,1.0,1.0,1.0,2.0,3.0,4.0
3,1010005476,92,8016597,53.0,53.0,4172.0,4172.0,1.0,1.0,1.0,2.0,3.0,4.0
4,1010005510,92,92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
